## Introduction
I will introduce a simple method using lightGBM as a starter.

## import
Load the necessary libraries.

In [ ]:
import os
import pandas as pd
from kaggle.competitions import nflrush
import numpy as np
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import KFold
import lightgbm as lgb
import gc
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
import tqdm

## train data
The shape of train data is 509762 × 49.
But, since one set consists of 22 lines, the actual number of data is 23171.
I converted it to a format that is easy to use.

In [ ]:
env = nflrush.make_env()
train_df = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2020/train.csv', low_memory=False)

In [ ]:
unused_columns = ["GameId","PlayId","Team","Yards","TimeHandoff","TimeSnap"]

In [ ]:
#将不在unused_columns中的且每个球员会有不同数值的数据视为unique_column
unique_columns = []
for c in train_df.columns:
    if c not in unused_columns+["PlayerBirthDate"] and len(set(train_df[c][:11]))!= 1:
        unique_columns.append(c)
        print(c," is unique")
unique_columns+=["BirthY"]

In [ ]:
#判断数据是否按照playID-Team来排序
ok = True
for i in range(0,509762,22):
    p=train_df["PlayId"][i]
    for j in range(1,22):
        if(p!=train_df["PlayId"][i+j]):
            ok=False
            break
print("train data is sorted by PlayId." if ok else "train data is not sorted by PlayId.")
ok = True
for i in range(0,509762,11):
    p=train_df["Team"][i]
    for j in range(1,11):
        if(p!=train_df["Team"][i+j]):
            ok=False
            break
print("train data is sorted by Team." if ok else "train data is not sorted by Team.")

Since the training data was sorted, preprocessing can be done easily.

In [ ]:
#将（每一行的数据都相同）非特殊列合并起来，并将特殊列逐一添加到末尾（对其中三列需要在之后作特殊处理，因此不在此处合并）
all_columns = []
for c in train_df.columns:
    if c not in unique_columns + unused_columns+["DefensePersonnel","GameClock","PlayerBirthDate"]:
        all_columns.append(c)
all_columns.append("DL")
all_columns.append("LB")    
all_columns.append("DB")
all_columns.append("GameHour")   
for c in unique_columns:
    for i in range(22):
        all_columns.append(c+str(i))

In [ ]:
lbl_dict = {}
for c in train_df.columns:
    if c == "DefensePersonnel":#记录防守人员配置（每个位置上的防守人员数）
        arr = [[int(s[0]) for s in t.split(", ")] for t in train_df["DefensePersonnel"]]
        train_df["DL"] = np.array([a[0] for a in arr])
        train_df["LB"] = np.array([a[1] for a in arr])
        train_df["DB"] = np.array([a[2] for a in arr])
    elif c == "GameClock":#比赛时间仅记录当前的小时（主要用来区分比赛进行在一天内的哪一个时间段）
        arr = [[int(s) for s in t.split(":")] for t in train_df["GameClock"]]
        train_df["GameHour"] = pd.Series([a[0] for a in arr])
    elif c == "PlayerBirthDate":#球员生日仅记录其出生年（主要用于代表球员年龄）
        arr = [[int(s) for s in t.split("/")] for t in train_df["PlayerBirthDate"]]
        train_df["BirthY"] = pd.Series([a[2] for a in arr])
    # elif c == "PlayerHeight":
    #     arr = [float(s.split("-")[0]) * 30.48 + float(s.split("-")[1]) * 2.54
    #         for s in list(train_df["PlayerHeight"])]
    #     train_df["PlayerHeight"] = pd.Series(arr)
    elif train_df[c].dtype=='object' and c not in unused_columns: 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train_df[c].values))
        lbl_dict[c] = lbl
        train_df[c] = lbl.transform(list(train_df[c].values))


In [ ]:
#删去不需要的行
drop_columns = ["Team", "GameClock", "DefensePersonnel", "TimeHandoff", "TimeSnap", "PlayerBirthDate"]
train_df = train_df.drop(drop_columns, axis=1)#丢弃列

In [ ]:
#将22行数据压缩到一行内
train_data=np.zeros((509762//22,len(all_columns)))
for i in tqdm.tqdm(range(0,509762,22)):
    count=0
    for c in all_columns:
        if c in train_df:
            train_data[i//22][count] = train_df[c][i]
            count+=1
    for c in unique_columns:
        for j in range(22):
            train_data[i//22][count] = train_df[c][i+j]
            count+=1        

In [ ]:
#取出训练用的标签，并装好训练数据
y_train_ = np.array([train_df["Yards"][i] for i in range(0,509762,22)])
X_train = pd.DataFrame(data=train_data,columns=all_columns)
train_data_df = pd.DataFrame(data=train_data)
train_data_df.to_csv("train_data_cdropped.csv")

In [ ]:
data = [0 for i in range(199)]
for y in y_train_:#将其转换成概率标签
    data[int(y+99)]+=1
plt.plot([i-99 for i in range(199)],data)

Since the variance is small, I standardized the objective variable.

In [ ]:
X_train.head

In [ ]:
try_clf = 1#选择跑策略树算法（0）还是跑回归模型神经网络算法（1）还是跑分类模型神经网络算法（2）

In [ ]:
# scaler = preprocessing.StandardScaler()
# scaler.fit([[y] for y in y_train_])
# y_train = np.array([y[0] for y in scaler.transform([[y] for y in y_train_])])
#对数据进行标准化

scaler = preprocessing.StandardScaler()
scaler.fit(y_train_.reshape(-1, 1))#获得均值和方差
y_train = scaler.transform(y_train_.reshape(-1, 1)).flatten()

scaler1 = preprocessing.StandardScaler()
scaler1.fit(X_train)#获得均值和方差
X_train = pd.DataFrame(data=scaler1.transform(X_train))

## train
I used LGBMRegressor.
I wanted to use multi-class classification, but the number of datasets was small and it was difficult to split them including all labels.

In [ ]:
y_valid_pred = np.zeros(X_train.shape[0])
models = []
folds = 10
seed = 222
if try_clf == 0:
    kf = KFold(n_splits = folds, shuffle = True, random_state=seed)


    for tr_idx, val_idx in kf.split(X_train, y_train):#分割数据集、校验集
        tr_x, tr_y = X_train.iloc[tr_idx,:], y_train[tr_idx]
        vl_x, vl_y = X_train.iloc[val_idx,:], y_train[val_idx]

        print(len(tr_x),len(vl_x))
        tr_data = lgb.Dataset(tr_x, label=tr_y)
        vl_data = lgb.Dataset(vl_x, label=vl_y)  
        clf = lgb.LGBMRegressor(n_estimators=200,learning_rate=0.01)#使用回归模型
        clf.fit(tr_x, tr_y,
            eval_set=[(vl_x, vl_y)],
            early_stopping_rounds=20,
            verbose=False)
        y_valid_pred[val_idx] += clf.predict(vl_x, num_iteration=clf.best_iteration_)
        models.append(clf)

    gc.collect()
elif try_clf == 1:
    from sklearn import neural_network
    X_train.head
    X_train[np.isnan(X_train) == True] = 0#把数据集里面的NaN数据全部替换为0
    kf = KFold(n_splits = folds, shuffle = True, random_state=seed)

    r_y_train = scaler.inverse_transform(y_train)
    for tr_idx, val_idx in kf.split(X_train, r_y_train):#分割数据集、校验集
        tr_x, tr_y = X_train.iloc[tr_idx,:], r_y_train[tr_idx]
        vl_x, vl_y = X_train.iloc[val_idx,:], r_y_train[val_idx]

        print(len(tr_x),len(vl_x))
        mlp = neural_network.MLPRegressor(hidden_layer_sizes=(256, 128), activation="logistic",
                     solver='adam', alpha=0.0001,
                     batch_size='auto', learning_rate="adaptive",
                     learning_rate_init=0.001,
                     power_t=0.5, max_iter=200,tol=1e-4)
        mlp.fit(tr_x, tr_y)
        
        y_valid_pred[val_idx] += scaler.transform(mlp.predict(vl_x).reshape(-1, 1)).flatten()
        
        y_p = mlp.predict(vl_x)
        
        #计算评估函数（局部结果）
        y_pred = np.zeros((len(val_idx),199))
        y_ans = np.zeros((len(val_idx),199))
        for i,p in enumerate(np.round(y_p)):
            p+=99
            for j in range(199):
                if j>=p+10:
                    y_pred[i][j]=1.0
                elif j>=p-10:
                    y_pred[i][j]=(j+10-p)*0.05

        for i,p in enumerate(r_y_train[val_idx]):
            p+=99
            for j in range(199):
                if j>=p:
                    y_ans[i][j]=1.0
        print("validation score:",np.sum(np.power(y_pred-y_ans,2))/(199*(len(val_idx))))
        #计算评估函数（局部结果）
        
        
        #for i in range(100):
        #    print(y_p[i], r_y_train[i])
        mlp_loss_curve = mlp.loss_curve_
        plt.plot(range(1, len(mlp_loss_curve)+1), mlp_loss_curve)
        plt.show()
        models.append(mlp)

    gc.collect()
elif try_clf == 2:
    from sklearn import neural_network
    X_train.head
    X_train[np.isnan(X_train) == True] = 0#把数据集里面的NaN数据全部替换为0
    kf = KFold(n_splits = folds, shuffle = True, random_state=seed)

    r_y_train = scaler.inverse_transform(y_train)
    tr_y_1 = np.zeros((len(r_y_train), 199))
    for i, d in enumerate(r_y_train):
        for j in range(199):
            if j == d:
                tr_y_1[i][j] = 1
    
    print("begin training...")
    for tr_idx, val_idx in kf.split(X_train, r_y_train):#分割数据集、校验集
        tr_x, tr_y = X_train.iloc[tr_idx,:], r_y_train[tr_idx].astype("int")
        vl_x = X_train.iloc[val_idx,:]
        vl_y = tr_y_1[val_idx]
            
        print(len(tr_x),len(vl_x))
        mlp = neural_network.MLPClassifier(hidden_layer_sizes=(256, 128), activation="logistic",
                     solver='adam', alpha=0.0001,
                     batch_size='auto', learning_rate="adaptive",
                     learning_rate_init=0.001,
                     power_t=0.5, max_iter=200,tol=1e-4)
        mlp.fit(tr_x, tr_y)
        y_pred = mlp.predict_proba(vl_x)
        for i in range(50):
            print("pred:", y_pred[i,:])
            print("truth:", vl_y[i,:])
        print("validation score:",np.sum(np.power(y_pred-vl_y,2))/(199*(len(val_idx))))#评估（局部）
        mlp_loss_curve = mlp.loss_curve_
        plt.plot(range(1, len(mlp_loss_curve)+1), mlp_loss_curve)
        plt.show()
        
    gc.collect()

## evaluation
Continuous Ranked Probability Score (CRPS) is derived based on the predicted scalar value.
The CRPS is computed as follows:
$$
C=\frac{1}{199N}\sum_{m=1}^N\sum_{n=-99}^{99}(P(y\geq n)-H(n-Y_m))^2
$$
$H(x)=1$ if $x\geq 0$ else $0$

In [ ]:
r_y_valid_pred = scaler.inverse_transform(y_valid_pred)
r_y_train = scaler.inverse_transform(y_train)
for i in range(100):
    print(r_y_valid_pred[i], r_y_train[i])

In [ ]:
y_pred = np.zeros((509762//22,199))
y_ans = np.zeros((509762//22,199))
for i,p in enumerate(np.round(scaler.inverse_transform(y_valid_pred))):
    p+=99
    for j in range(199):
        if j>=p+10:
            y_pred[i][j]=1.0
        elif j>=p-10:
            y_pred[i][j]=(j+10-p)*0.05

for i,p in enumerate(scaler.inverse_transform(y_train)):
    p+=99
    for j in range(199):
        if j>=p:
            y_ans[i][j]=1.0
print("validation score:",np.sum(np.power(y_pred-y_ans,2))/(199*(509762//22)))

In [ ]:
y_pred_demo = np.zeros(199)
y_pred_sigmoid_demo = np.zeros(199)
p = 5 + 99

for j in range(199):
    y_pred_sigmoid_demo[j] = 1 / (1 + np.exp((p-j)/3))
plt.plot(range(199), y_pred_sigmoid_demo)

## make submission

When there is a label that does not exist in the training data, it is handled as nan.
If you can check the error one by one and complement it, you will get better score.

In [ ]:
index = 0
for (test_df, sample_prediction_df) in tqdm.tqdm(env.iter_test()):
    for c in test_df.columns:
        if c == "DefensePersonnel":
            try:
                arr = [[int(s[0]) for s in t.split(", ")] for t in test_df["DefensePersonnel"]]
                test_df["DL"] = [a[0] for a in arr]
                test_df["LB"] = [a[1] for a in arr]
                test_df["DB"] = [a[2] for a in arr]
            except:
                test_df["DL"] = [np.nan for i in range(22)]
                test_df["LB"] = [np.nan for i in range(22)]
                test_df["DB"] = [np.nan for i in range(22)]
        elif c == "GameClock":
            try:
                arr = [[int(s) for s in t.split(":")] for t in test_df["GameClock"]]
                test_df["GameHour"] = pd.Series([a[0] for a in arr])
            except:
                test_df["GameHour"] = [np.nan for i in range(22)]
        elif c == "PlayerBirthDate":
            try:
                arr = [[int(s) for s in t.split("/")] for t in test_df["PlayerBirthDate"]]
                test_df["BirthY"] = pd.Series([a[2] for a in arr])
            except:
                test_df["BirthY"] = [np.nan for i in range(22)]
        # elif c == "PlayerHeight":
        #     try:
        #         arr = [float(s.split("-")[0]) * 30.48 + float(s.split("-")[1]) * 2.54
        #             for s in list(test_df["PlayerHeight"])]
        #         test_df["PlayerHeight"] = pd.Series(arr)
        #     except:
        #         test_df["PlayerHeight"] = [np.nan for i in range(22)]
        elif c in lbl_dict and test_df[c].dtype=='object'and c not in unused_columns\
            and not pd.isnull(test_df[c]).any():
            try:
                test_df[c] = lbl_dict[c].transform(list(test_df[c].values))
            except:
                test_df[c] = [np.nan for i in range(22)]
    #删去不需要的行
    #drop_columns = ["GameClock", "DefensePersonnel", "TimeHandoff", "TimeSnap", "PlayerBirthDate"]
    #test_df.drop(drop_columns, axis=1)#丢弃列
    count=0
    test_data = np.zeros((1,len(all_columns)))

    for c in all_columns:
        if c in test_df:
            try:
                test_data[0][count] = test_df[c][index]
            except:
                if try_clf:
                    test_data[0][count] = np.nan
                else:
                    test_data[0][count] = 0
            count+=1
    for c in unique_columns:
        for j in range(22):
            try:
                test_data[0][count] = test_df[c][index + j]
            except:
                if try_clf:
                    test_data[0][count] = np.nan
                else:
                    test_data[0][count] = 0
            count+=1
    #test_data[np.isnan(test_data) == True] = 0
    if try_clf == 1:
        test_data = scaler1.transform(test_data)
    elif try_clf == 2:
        test_data = scaler1.transform(test_data)
    y_pred = np.zeros(199)
    
    #两个模型的输出形式不同，因此需要不同的处理方式
    if try_clf == 0 or try_clf == 1:
        if try_clf == 0:
            y_pred_p = np.sum(np.round(scaler.inverse_transform(
                [model.predict(test_data)[0] for model in models])))/folds
        elif try_clf == 1:
            y_pred_p = np.sum(np.round([model.predict(test_data)[0] for model in models]))/folds
        y_pred_p += 99
        for j in range(199):
            if j>=y_pred_p+10:
                y_pred[j]=1.0
            elif j>=y_pred_p-10:
                y_pred[j]=(j+10-y_pred_p)*0.05
    env.predict(pd.DataFrame(data=[y_pred],columns=sample_prediction_df.columns))
    index += 22
env.write_submission_file()

The organizers seemed to expect to predict one by one, so I did. 
However, it seems that it is likely to be faster to predict at once after all the evaluation data is acquired by dummy input.


This model is a simple one that has not been tuned, so I think we can still expect a better score.
Please let me know if you have any opinions or advice.